In [53]:
import re

import black
import jupyter_black
from math import prod
from parse import parse

jupyter_black.load(lab=True, target_version=black.TargetVersion.PY310)


def ints(text: str) -> list[int]:
    return [int(x) for x in re.findall("-?\d+", text)]


def first(iterable):
    return next(iter(iterable))


def data(day: int, parser=str, sep="\n", example=False) -> list:
    "Split the day's input file into sections separated by `sep`, and apply `parser` to each."
    filename = f"2023/{day}-example.txt" if example else f"2023/{day}.txt"
    sections = open(filename).read().rstrip().split(sep)
    return [parser(section) for section in sections]

In [94]:
# Day 1: Trebuchet?!
numbers = {
    "one": "1",
    "two": "2",
    "three": "3",
    "four": "4",
    "five": "5",
    "six": "6",
    "seven": "7",
    "eight": "8",
    "nine": "9",
}


def calibration_value(line, part1=True):
    numbers_found = ""
    for start, char in enumerate(line):
        if char.isdigit():
            numbers_found += char
        if part1:
            continue
        for key, value in numbers.items():
            if line.startswith(key, start):
                numbers_found += value
    calibration_value = numbers_found[0] + numbers_found[-1]
    return int(calibration_value)


games = data(1)
result = sum([calibration_value(line) for line in games])
print(f"Part 1: {result}")  # 53921
result = sum([calibration_value(line, part1=False) for line in games])
print(f"Part 2: {result}")  # 54676

Part 1: 53921
Part 2: 54676


In [55]:
# Day 2: Cube Conundrum
games = data(2)
possible_games = []
powers = []
for game_nr, game in enumerate(games, 1):
    max_cubes_seen = {"red": 0, "blue": 0, "green": 0}
    cubes_seen = game.split(": ")[1].split("; ")
    for cubes in cubes_seen:
        for cube in cubes.split(", "):
            count, color = cube.split()
            count = int(count)
            if count > max_cubes_seen[color]:
                max_cubes_seen[color] = count
    if (
        max_cubes_seen["red"] <= 12
        and max_cubes_seen["green"] <= 13
        and max_cubes_seen["blue"] <= 14
    ):
        possible_games.append(game_nr)
    powers.append(prod(max_cubes_seen.values()))

print(f"Part 1: {sum(possible_games)}")  # 2563
print(f"Part 2: {sum(powers)}")  # 70768

Part 1: 2563
Part 2: 70768
